# Atividade avaliativa para a disciplina DMMLII
## Aluno: Hermes Araujo

In [ ]:
# Importando todas as bibliotecas

import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scikitplot as skplt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier

In [ ]:
#Carregando o banco
df = pd.read_csv('/kaggle/input/hmeq-data/hmeq.csv')
df.head(20)

## Dicionário dos dados

**VARIÁVEL** - TIPO - DESCRIÇÃO
<br/>**BAD** - numérico - assume 1 se o cliente não cummpriu com o empréstimo e 0 se cumpriu com o pagamento 
<br/>**LOAN** - numérico - valor total da dívida que o cliente quer contrair
<br/>**MORTDUE** - numérico - valor atual da hipoteca
<br/>**VALUE** - numérico - valor da propriedade hipotecada
<br/>**REASON** - categórico - razão da hipoteca. Se DebtCon é consolidação de dívida e se HomeImp e para melhoria da propriedade
<br/>**JOB** - categórico - emprego do devedor. Seis categorias próprias
<br/>**YOJ** - numérico - anos no emprego
<br/>**DEROG** - numérico - número de relatórios de inconformidades
<br/>**DELINQ** - numérico - número de linhas de crédito inadimplentes
<br/>**CLAGE** - númerico - idade da linha de crédito mais antiga em meses
<br/>**NINQ** - numérico - número de linhas de crédito recentes
<br/>**CLNO** numérico - total de linhas de crédito
<br/>**DEBTINC** - númerico - razão entre o débito e as entradas do cliente

## Análise exploratória

In [ ]:
#verificando se o tipo dos dados no banco está em conformidade com o dicionário
df.info()

In [ ]:
#verificando a quantidade de valores faltantes dos campos
df.isnull().sum()

In [ ]:
#Boxplots dos valores das dívidas contraídas em função da razão da hipoteca. Estão comparados pelo pagamento ou não.
sns.boxplot(x="REASON", y="LOAN",
            hue="BAD", palette=["m", "g"],
            data=df)
sns.despine(offset=10, trim=True)

Percebemos que os inadimplentes pegaram empréstimos em valores menores quando o motivo era melhoria da propriedade. Já no motivo consolidação de débito, a massa de dados é bastante parecida entre os pagadores, com os inadimplentes tendo ainda uma mediana um pouco menor. Em todos os boxplots percebemos grandes quantidades de outliers a maior.

In [ ]:
#Scatterplot do valor da propriedade com o valor atual da hipoteca
plt.plot(range(500000))
ax = sns.scatterplot(x="VALUE", y="MORTDUE",
                     hue="BAD",
                     data=df)
plt.xlim(0, 500000)
plt.ylim(0, 500000)
plt.gca().set_aspect('equal', adjustable='box')
plt.draw()

Todas as observações acima da linha identidade possuem um valor de hipoteca maior que o próprio valor da propriedade hipotecada. Curiosamente, a maioria não é "má pagadora".

## Modelagem


In [ ]:
#Embonecando as variáveis categóricas
df = pd.get_dummies(df, columns=['REASON','JOB'])
df.head().T

In [ ]:
#Definindo as variáveis independentes
feats = [c for c in df.columns if c not in ['BAD']]

In [ ]:
#Separando a base em treino e teste
train, valid = train_test_split(df, test_size=0.2, random_state=42)

train.shape, valid.shape

Nesse caso não temos base de teste verdadeira (sem a variável dependente), então separaremos a base somente em treino e validação

In [ ]:
#Rodando o primeiro modelo
rf = RandomForestClassifier(n_estimators=200, random_state=42)
rf.fit(train[feats], train['BAD'])

Como verificamos lá em cima, existem muitos valores faltantes na base e o modelo não vai funcionar dessa forma. Partiremos então para a imputação.

In [ ]:
#Imputação
df.fillna(-1, inplace=True)
df.isnull().sum()

In [ ]:
#Refazendo a separação
train, valid = train_test_split(df, test_size=0.2, random_state=42)

train.shape, valid.shape

In [ ]:
#Rodando o primeiro modelo novamente
rf = RandomForestClassifier(n_estimators=200, random_state=42)
rf.fit(train[feats], train['BAD'])

In [ ]:
#Aplicando o modelo na base de validação e verificando a acurácia
preds_val = rf.predict(valid[feats])

accuracy_score(valid['BAD'], preds_val)

In [ ]:
#Mostrando a matriz de confusão
skplt.metrics.plot_confusion_matrix(valid['BAD'],preds_val)

**Vemos na matriz de confusão os seguintes dados:**
<br/>**Sensibilidade:** 0,72 (190/(190+75)) -> Capacidade de acertar os positivos
<br/>**Especificidade:** 0,97 (897/(897+30)) -> Capacidade de acertar os negativos
<br/>**Acurácia:** 0,91 (897+190)/1192) -> Número de acertos totais frente ao tamanho do teste
<br/><br/>Surpreendentemente parece se tratar de um bom modelo, mas vamos tentar melhorá-lo ainda mais.

In [ ]:
#Testando o limitador de profundidade da árvore
for i in range(1,11,1):
    rft = RandomForestClassifier(n_estimators=200, random_state=42, max_depth=i)
    rft.fit(train[feats], train['BAD'])
    pred_teste = rft.predict(valid[feats])
    print(str(i)+" de profundidade: "+str(accuracy_score(valid['BAD'], pred_teste)))

Vemos que a profundidade da árvore não melhora o modelo. Limitá-la diminuiu a acurácia em comparação com o nosso valor original

In [ ]:
#Testando o número de estimadores
for i in range(1000,100,-100):
    rft = RandomForestClassifier(n_estimators=i, random_state=42)
    rft.fit(train[feats], train['BAD'])
    pred_teste = rft.predict(valid[feats])
    print(str(i)+": "+str(accuracy_score(valid['BAD'], pred_teste)))

O número de estimadores já contribui alguma coisa com o modelo. Com 1000 estimadores temos uma melhora de 0,0008 na acurácia. Mas o desempenho não é linear como na profundidade. Ao subir de 200 para 300 temos uma piora, fica um tempo estagnado, em 800 temos o valor inicial novamente e uma pequena melhora em 1000.

In [ ]:
#Verificando o desbalanceio da variável dependente
df['BAD'].value_counts()

O valor de mal pagadores é de 20% dos registros (1189/5960). Vamos testar aplicar pesos diferentes para esses registros

In [ ]:
#Testando colocar pesos nas possibilidades pagadores para atacar o desbalanceio
class_weight = dict({1:4, 0:1})
rdf = RandomForestClassifier(bootstrap=True,
            class_weight=class_weight, 
            criterion='gini',
            max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=4, min_samples_split=10,
            min_weight_fraction_leaf=0.0, n_estimators=200,
            oob_score=False,
            random_state=42,
            verbose=0, warm_start=False)

rdf.fit(train[feats], train['BAD'])

pred_teste = rdf.predict(valid[feats])
print(accuracy_score(valid['BAD'], pred_teste))

Foram testados diversos pesos, mas o máximo de acurácia alcançada com o peso 3 para o mau pagador foi a mesma do modelo original. Deixei com peso 4 para ilustrar que o esforço não adiantou muito.

## Cross Validation

Em vez de dividir em treino e teste como fizemos, poderiamos ter feito o cross validation logo de cara, para ter uma noção da qualidade do modelo pretendido e até mesmo comparar dois tipos de modelos diferentes. Vamos ver essa comparação entre o random forest e o XGB

In [ ]:
# cria o vetor de notas, mostra e mostra a média
scores = cross_val_score(rf, df[feats], df['BAD'], n_jobs=-1, cv=5)

scores, scores.mean()

Podemos ver que através do cross validation conseguimos modelos com acurácias piores do que a separação que foi feita com o seed 42. Demos sorte! Vamos testar agora cross validation + xgb

In [ ]:
# cria um objeto xgb
xgb = XGBClassifier(n_estimators=200, n_jobs=-1, random_state=42, learning_rate=0.05)

In [ ]:
#Usa o cross validation como antes, mas com o xgb
scores = cross_val_score(xgb, df[feats], df['BAD'], n_jobs=-1, cv=5)

scores, scores.mean()

Os desempenhos do xgb não foram tão bons quanto da random forest

## Grid Search

Uma alternativa a ter mexido nos parâmetros na mão como fizemos lá em cima é o Grid Search. Vamos demonstrá-lo abaixo com o random forest.

In [ ]:
#Cria um dicionário com os tipos de parâmetros que serão testados
grid_param = {
    'n_estimators': [100, 300, 500, 800, 1000],
    'criterion': ['gini', 'entropy'],
    'bootstrap': [True, False]
}

In [ ]:
#Cria o objeto grid search utilizando o dicionário anterior
gd_sr = GridSearchCV(estimator=rf,
                     param_grid=grid_param,
                     scoring='accuracy',
                     cv=5,
                     n_jobs=-1)

In [ ]:
#Treina o modelo testando combinações de todos os parâmetros (demorado)
gd_sr.fit(df[feats], df['BAD'])

In [ ]:
#Mostra os melhores parâmetros
best_parameters = gd_sr.best_params_
print(best_parameters)

In [ ]:
#Mostra a acurácia com os melhores parâmetros
best_result = gd_sr.best_score_
print(best_result)

## Conclusão

   Num próximo modelo podemos utilizar o cross validation logo de cara para decidir que tipo de modelo usar e ter uma ideia da média de acurácia dos modelos mesmo sem separar a base. Além disso, para melhorar o modelo podemos testar várias combinações de parâmetros com o Grid Search até achar os melhores parâmetros. Logicamente falando essa seria a maneira mais organizada de fazer as coisas. Entretanto foi bom ter feito a separação antes de tudo isso. Em primeiro lugar, porque ilustra o caminho de aprendizado da disciplina, mas principalmente porque mostra o poder que uma separação feita de determinada forma tem na modelagem (provavelmente mostra na realidade a força do overfitting). 

   Outros parâmetros poderiam ter sido testados no grid search, como class_weight, max_features, max_leaf_nodes, min_impurity_decrease, min_impurity_split, min_samples_leaf, min_samples_split, min_weight_fraction_leaf, etc.
    
   De qualquer forma, para todos os efeitos, o modelo de random forest gerado com a base separada foi o que desempenhou melhor, com impressionantes 91,2% de acurácia.